<a href="https://colab.research.google.com/github/farzadgolnoori/test/blob/main/SignalProcessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Mount Drive You can use blew command if you use local computer not colab
DIR = os.path.abspath('./dataset')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Get data, labels and create dataset
 

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 

DIR = os.path.abspath('/content/drive/My Drive/RadarProject/fulldataset')

LABEL_MAPPER = {'Cars': 0, 'Drones': 1, 'People': 2}

INV_LABEL_MAPPER = {v: k for k, v in LABEL_MAPPER.items()}

#DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

def get_data_for_label(label: str):
    X, y = [], []
    for root, dirs, files in os.walk(os.path.join(DIR, label)):
        for file in files:
            if file.endswith('.csv'):
                y.append(LABEL_MAPPER[label])
                df = pd.read_csv(os.path.join(root, file), sep=',', header=None)
                X.append(df.values)
    print(f'Loaded {len(y)} examples for label {label} encoded with {LABEL_MAPPER[label]}')
    return X, y

X_cars, y_cars = get_data_for_label('Cars')
X_drones, y_drones = get_data_for_label('Drones')
X_people, y_people = get_data_for_label('People')

X = X_cars + X_drones + X_people
y = y_cars + y_drones + y_people
X, y = np.array(X), np.array(y)

Loaded 5720 examples for label Cars encoded with 0
Loaded 5065 examples for label Drones encoded with 1
Loaded 6700 examples for label People encoded with 2


Changing image size to 100*100 for transfer learning(to using pre-trained models your images should be 32*32 size)

# **Do Not use this section if you don't use transfer learning**

In [ ]:
import numpy as np
from PIL import Image

plt.show()
Xresized=[]
for pic in X:
  resizedImage=np.array(Image.fromarray(pic).resize((100, 100))).astype('float32')
  Xresized.append(resizedImage)
Xresized
print('finished')
Xresized=np.array(Xresized)
Xresized.shape

finished


(17485, 100, 100)

Changing image channel from 1 to 3 (pre-trained models accept just RGB images because they trained on imagenet dataset.

# **Do Not use this section if you don't use transfer learning**

In [ ]:
Xresized=np.repeat(Xresized[..., np.newaxis], 3, -1)
print(Xresized.shape)  # 

(17485, 100, 100, 3)


Showing **imges** 11 61 (orginal) and 100 100 in a row

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image
resizedImage=np.array(Image.fromarray(X[0]).resize((100, 100))).astype('float32')
fig, axs =plt.subplots(2, 2, figsize=(10, 10))
axs[0, 0].imshow(X[0], cmap='jet', vmin=-140, vmax=-70)
axs[0, 1].imshow(resizedImage, cmap='jet', vmin=-140, vmax=-70)

Dividing Dataset to train, test and validation subsets

**If you don't use transfer learning instead of Xresized use X array.**

In [ ]:
from sklearn.model_selection import train_test_split

SEED = 0
val_size, test_size = 0.1, 0.1

# train-test split
X_trainval, X_test, y_trainval, y_test = train_test_split(
    Xresized, y, test_size=test_size, random_state=SEED, stratify=y
)

# train-validation split
X_train, X_val, y_train, y_val = train_test_split(
    X_trainval,
    y_trainval,
    test_size=val_size / (1 - test_size),
    random_state=SEED,
    stratify=y_trainval,
)

print(X_train.shape)

print(X_val.shape)

print(X_test.shape)

(13987, 100, 100, 3)
(1749, 100, 100, 3)
(1749, 100, 100, 3)


First Custom Model(from scratch without using transfer learning)

In [ ]:
from keras.optimizers import Adam

from tensorflow.keras.optimizers import Adadelta,Adagrad,SGD,RMSprop,Adam,SGD

from tensorflow.keras.layers import SeparableConv2D,Conv2D,MaxPooling2D,Flatten
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D,Dropout,Input
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import Callback

model = Sequential()


# Conv Layer 1
model.add(Conv2D(filters = 10, kernel_size=(3, 3), activation='relu',input_shape= (100,100,3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters = 20, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(64, activation='relu'))

model.add(Dense(64, activation='relu'))

model.add(Dense(3, activation='softmax'))

model.compile(loss = 'sparse_categorical_crossentropy',optimizer='adam', metrics = ['accuracy'])

print(model.summary())

#adam_opt=Adam(lr=0.0001, beta_1=0.6, beta_2=0.995, epsilon=None, decay=0.0, amsgrad=False)

#model.compile(loss = 'categorical_crossentropy',optimizer=adam_opt, metrics = ['accuracy'])

model.fit(X_train, y_train, epochs=50, batch_size=32,validation_data=(X_val, y_val), verbose=1, shuffle=True)


Transfer Learning _ Feature Extractor( mobile-netV2, VGG16, ResNet50, DenseNet201, Xception)

In [ ]:
from tensorflow.keras.applications import VGG16 
from tensorflow.keras.applications import DenseNet201
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.layers import SeparableConv2D,Conv2D,MaxPooling2D,Flatten
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D,Dropout,Input
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.applications import EfficientNetB4
from tensorflow.keras.optimizers import Adam

base_model = DenseNet201(weights='imagenet',include_top= False,input_shape=(100,100,3))

x = base_model.output

x = GlobalAveragePooling2D()(x)

x = Dense(64, activation='relu')(x)

predictions = Dense(3, activation= 'softmax')(x)

model = Model(base_model.input,predictions)

optimizer = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=True)

model.compile(loss='sparse_categorical_crossentropy',optimizer=optimizer, metrics=['accuracy'])

model.fit(X_train, y_train, epochs=30, batch_size=32,validation_data=(X_val,y_val), verbose=1)

scores= model.evaluate(X_test, y_test)
    
print(scores[1])


In [ ]:
from tensorflow.keras.applications import VGG16 
from tensorflow.keras.applications import DenseNet201
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.layers import SeparableConv2D,Conv2D,MaxPooling2D,Flatten
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D,Dropout,Input
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.applications import EfficientNetB4
from tensorflow.keras.optimizers import Adam

base_model = ResNet50(weights='imagenet',include_top= False,input_shape=(100,100,3))

x = base_model.output

x = GlobalAveragePooling2D()(x)

x = Dense(64, activation='relu')(x)

predictions = Dense(3, activation= 'softmax')(x)

model = Model(base_model.input,predictions)

optimizer = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=True)

model.compile(loss='sparse_categorical_crossentropy',optimizer=optimizer, metrics=['accuracy'])

model.fit(X_train, y_train, epochs=30, batch_size=32,validation_data=(X_val,y_val), verbose=1)

scores= model.evaluate(X_test, y_test)
    
print(scores[1])


94765736/94765736 [==============================] - 2s 0us/step


In [ ]:
scores= model.evaluate(X_test, y_test)
    
print(scores[1])

In [ ]:
    
model.save("/content/drive/My Drive/app/"+"signal_densenet"+'.h5')